In [1]:
'''
差分内容
- week dayofyer
0.36472
- storeテーブルのPromo2SinceWeek	Promo2SinceYear	PromoIntervalの情報以外を使用してtrainにMERGE

'''

'\n差分内容\n- week dayofyer\n0.36472\n- storeテーブルのPromo2SinceWeek\tPromo2SinceYear\tPromoIntervalの情報以外を使用してtrainにMERGE\n\n'

In [41]:
# main module
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import seaborn as sns

import lightgbm as lgb

from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error

import datetime
import gc
import logging
import pickle
from pickle import load
import sys, os

# -------------------------------------独自モジュール-------------------------------------
sys.path.append('../src/') #モジュールが入っているディレクトリのパスを指定
import eda
import maprepro as mpre
import maprepro2 as mpre2
# import config
# from utils import setup_logger, ModelFactory
# -------------------------------------独自モジュール-------------------------------------

In [42]:
path = '../../input'
sample = pd.read_csv(f'{path}/sample_submission.csv')
store = pd.read_csv(f'{path}/store.csv')

test = pd.read_csv(f'{path}/test.csv',
                       parse_dates=['Date'],
                       date_parser=(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d')))

train = pd.read_csv(f'{path}/train.csv',
                       parse_dates=['Date'],
                       date_parser=(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d')))


C:\Users\thyt\anaconda3\envs\rake4\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
# np.log1p(train.Sales).plot.hist(bins=100)
train.query("Open == 0").Sales.value_counts()

0    172817
Name: Sales, dtype: int64

In [44]:
train = train.sort_values('Date')

In [45]:
def mk_YMD_DOY(df):
    var_name = 'Date'
    df[var_name + 'Year'] = pd.Index(df[var_name]).year
    df[var_name + 'Month'] = pd.Index(df[var_name]).month
    df[var_name + 'Week'] = pd.Index(df[var_name]).week
    df[var_name + 'Day'] = pd.Index(df[var_name]).day
    df[var_name + 'DayOfYear'] = pd.Index(df[var_name]).dayofyear
    return df
train = mk_YMD_DOY(train)
test = mk_YMD_DOY(test)

# 時系列データであり、時間に沿って変数periodを設定したとする
def mk_period(df,testdata=False):
    if testdata == False:
        df['period'] = np.arange(0, len(df)) // (len(df) // 4)
        df['period'] = np.clip(df['period'], 0, 3)
        return df
    else:
        df['period'] = 4
        return df

train = mk_period(train)
test = mk_period(test,testdata=True)

C:\Users\thyt\AppData\Local\Temp/ipykernel_14240/3274496651.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df[var_name + 'Week'] = pd.Index(df[var_name]).week


In [46]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [47]:
def mk_cat(df):
    # intなので決定木系でないと使えない処理
    store['StoreType'] = store['StoreType'].astype('category').cat.codes
    store['Assortment'] = store['Assortment'].astype('category').cat.codes
    return store
store = mk_cat(store)
use = ['Store','StoreType','Assortment','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2']
train = pd.merge(train, store[use], how='left', on=['Store'])
test = pd.merge(test, store[use], how='left', on=['Store'])

In [48]:
train['StateHoliday'] = train.StateHoliday.astype('category')
test['StateHoliday'] = test.StateHoliday.astype('category')

alltmp = sorted(train.columns)
target = ['Sales']
notuse = ['Id','Date']
use = ['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday','period',
        'DateYear','DateMonth','DateWeek','DateDay']

# unknown--'Customers'column
use2 = ['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance',
        'CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2','DateYear','DateMonth','DateWeek','DateDay','DateDayOfYear','period']
        
categorical_features = ['StateHoliday']

train_y = train[target]
train_x = train[use2]
test_x = test[use2]

In [49]:
import warnings
warnings.simplefilter('ignore')
train_x = eda.reduce_mem_usage(train_x)

start size(BEFORE): 123.20 Mb
Mem. usage decreased to 32.98 Mb (AFTER:73.2% reduction)


In [69]:
# --------------------------------------------------
import maprepro2 as mpre2
NOW,TMP_DIR = mpre2.mk_dir()
gc.collect()
# --------------------------------------------------

va_period_list = [1, 2, 3]
for va_period in va_period_list:
    print('i ================================================================== ',va_period)
    is_tr = train_x['period'] < va_period
    is_va = train_x['period'] == va_period
    tr_x, va_x = train_x[is_tr], train_x[is_va]
    tr_y, va_y = train_y[is_tr], train_y[is_va]
    
    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータの設定
    params = {'objective': 'regression',
                'seed': 71,
                'verbose': 1,
                'metrics': 'rmse',
                'force_col_wise':'true' # メモリが足りないから
                }
    num_round = 100

    # 学習の実行
    categorical_features = categorical_features
    model = lgb.train(params, lgb_train, num_boost_round=num_round,
                    categorical_feature=categorical_features,
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval],
                    )


    tmpfile = f'{TMP_DIR}/trained_model{va_period}.pkl'
    pickle.dump(model, open(tmpfile, 'wb'))

    # バリデーションデータでのスコアの確認
    va_pred = model.predict(va_x); va_pred = va_pred.reshape(-1, 1)

    # Open=0の日はすべてSalesが0で、0以外を予測することはありえないので0で更新(モデルは自然数を予測しているみたいだが、
    # Open=0ならSales=0をモデルに教えられていないということ)
    tmp =va_y['Sales']==0
    va_pred[tmp]=0
    va_pred=va_pred+1; va_y=va_y+1

    # --------------------------------------------------
    # score = mean_squared_error(va_y, va_pred)
    RMSPE = np.sqrt(np.mean(((  (va_y-va_pred)/va_y)**2) )).values
    RMSPE = RMSPE.astype(float)[0] # printのために単一の数値にしたかった
    # score = log_loss(va_y, va_pred)
    print('RMSPE RMSPE RMSPE RMSPE RMSPE RMSPE: {:.4f}'.format(RMSPE))
    # --------------------------------------------------

    del model # 学習済みモデルを削除


i ==================================================================  1
[LightGBM] [Info] Total Bins 880
[LightGBM] [Info] Number of data points in the train set: 254302, number of used features: 16
[LightGBM] [Info] Start training from score 5576.839962
[1]	train's rmse: 3492.89	valid's rmse: 3626.56
[2]	train's rmse: 3298.58	valid's rmse: 3444.32
[3]	train's rmse: 3129.87	valid's rmse: 3285.79
[4]	train's rmse: 2984.19	valid's rmse: 3149.43
[5]	train's rmse: 2861.77	valid's rmse: 3035.07
[6]	train's rmse: 2755.81	valid's rmse: 2935.07
[7]	train's rmse: 2666.75	valid's rmse: 2852.04
[8]	train's rmse: 2589.89	valid's rmse: 2781.03
[9]	train's rmse: 2522.46	valid's rmse: 2719.04
[10]	train's rmse: 2466.54	valid's rmse: 2663.68
[11]	train's rmse: 2418.06	valid's rmse: 2620.77
[12]	train's rmse: 2372.58	valid's rmse: 2578.24
[13]	train's rmse: 2335.03	valid's rmse: 2542.37
[14]	train's rmse: 2305.2	valid's rmse: 2513.32
[15]	train's rmse: 2268.6	valid's rmse: 2482.1
[16]	train's rmse: 224

In [74]:
modeldict = mpre2.load_models(TMP_DIR)

load開始
load終わりました


In [83]:
pred_dict

{1: array([[ 6495.78620506],
        [ 6407.08995631],
        [ 8742.68250883],
        ...,
        [ 5550.3949349 ],
        [17420.17054874],
        [ 5428.42836637]]),
 2: array([[ 6087.100284  ],
        [ 6686.49205925],
        [ 8529.82967159],
        ...,
        [ 5804.27020875],
        [18805.87994394],
        [ 5628.94142969]]),
 3: array([[ 6127.47238877],
        [ 7023.28959974],
        [ 8956.93612206],
        ...,
        [ 6213.77563361],
        [18508.17462156],
        [ 5806.0774262 ]])}

In [81]:
pred_dict = mpre2.mk_pred_dict(modeldict,test_x)

予測リストが入った辞書作成開始
予測リストが入った辞書作成終了


In [162]:
pred_wei_average = mpre2.wei_average(pred_dict,len(test_x))

複数モデルを使用した重み付き平均予測を開始します
重み付き平均pred_wei_averageがreturnされます
重み付き平均pred_wei_averageがreturnされました


In [163]:
subpred = pd.DataFrame(pred_wei_average)
# subpred.sort_values('Sales')

sub = pd.concat([test['Id'],subpred],axis=1)
sub.columns = ['Id','Sales']

In [164]:
mpre2.mk_output(df=sub,NOW=NOW,PRACTICE=False)